#Matthew Keiser Homework #2

#Conceptual

##1) What is the difference between train and test set?

A training set is used to develop a model, and a test set is used to assess the predictive capability of the model developed with the training set.

##2) Why is it important to have a seperate data set for training?

A model developed with a set of data can simply memorize the data set for a 100% accurate model. By separating the original data set into a training and testing set, assuming the test set is selected randomly, the actual predictive capability of the model.

##3) What are some advantages and disadvantages of the KNN algorithm?

KNN is a simple, quick running algorithm that represents the variability of the entire dataset. It is limited in the number of features that can be used; if there are too many features, all points of data will be approximately equally far from each other, meaning there will be very little variability in the predicted output. Also, the data must be normalized for this algorithm to work.

##4) Explain in your own words the bias variance tradeoff

The output of a less complex model approaches the mean of the data; a less complex model is thus biased toward a single output, and exibits a great deal of error with regard to both the training and testing data sets. A more complex model effectively memorizes the training data set; a more complex model thus results in a large amount of variance, and exibits a great deal of error with regard to the testing data set but little with regard to the training data set.

##5) Do you think KNN suffers from high bias or high variance?

When K equals the length of the training dataset, KNN suffers from high bias beacuse it returns the average value of the training dataset, regardless of the input. When K equals one, KNN suffers from high varience because the result is simply the closest point in the training dataset to the input.

##6) List some possible solutions when suffering from high variance.

Variance is the result of an over complex model; therefore one remedy is to reduce the complexity - remove features or implement regularization (Ridge or Lasso) when using linear regression; increase K when using KNN.

##7) List some possible solutions when suffereing from high bias.

Bias is the result of an under-complex model; therefore, one remedy is to increase complexity in the model - add features (polynomial or interaction) in linear regression; reduce K when using KNN. Another remedy is to add data - additional observations or additional features.

##8) Would you choose K when doing K Nearest Neighbors?

K is an input for KNN, so a K must be chosen to run the model. However, cross-validation may be used to determine the best (lowest test error) K.

#Practical

##Initialize Modules and Red in the Dataset

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn import cross_validation
from sklearn import neighbors
from sklearn import metrics
from sklearn import grid_search
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/galvin-mj/DAT_ATL_15/master/Datasets/ProstateCancer.csv')
del data['Unnamed: 0']
data.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [3]:
train_x = data[data['train'] == 'T'].drop('train', 1).drop('lpsa', 1)
scaled_train_x = sk.preprocessing.scale(train_x)
train_y = data['lpsa'][data['train'] == 'T']
scaled_train_y = sk.preprocessing.scale(train_y)
test_x = data[data['train'] == 'F'].drop('train', 1).drop('lpsa', 1)
scaled_test_x = sk.preprocessing.scale(test_x)
test_y = data['lpsa'][data['train'] == 'F']
scaled_test_y = sk.preprocessing.scale(test_y)

In [4]:
data.corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
lcavol,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652,0.734460
lweight,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354,0.433319
age,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112,0.169593
lbph,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460,0.179809
svi,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648,0.566218
lcp,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528,0.548813
gleason,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905,0.368987
pgg45,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000,0.422316
lpsa,0.734460,0.433319,0.169593,0.179809,0.566218,0.548813,0.368987,0.422316,1.000000


##1) Perform regression on the prostate data set. Your goal is to minimize the mean squared error (MSE). Split this into a several parts and at a minimum utilize

###Linear Regression

In [35]:
sk_linreg_model = sk.linear_model.LinearRegression()
sk_linreg_model.fit(scaled_train_x,scaled_train_y)
linreg_error = metrics.mean_squared_error(scaled_test_y,sk_linreg_model.predict(scaled_test_x))
linreg_MSE = ['Linear Regression',linreg_error]

print('SkLearn Linear Regression Mean Squared Error is',linreg_error)
print('SkLearn Linear Regression Intercept is',sk_linreg_model.intercept_)
print('SkLearn Linear Regression Model Coefficients Are:')
print([i for i in zip(sk_linreg_model.coef_,train_x.keys())])

SkLearn Linear Regression Mean Squared Error is 0.491127418061
SkLearn Linear Regression Intercept is 1.43901040001e-16
SkLearn Linear Regression Model Coefficients Are:
[(0.59314448426023669, 'lcavol'), (0.24229135512063388, 'lweight'), (-0.11802302729236888, 'age'), (0.17553030474789702, 'lbph'), (0.25634746039416223, 'svi'), (-0.2392803027906078, 'lcp'), (-0.017315211482775372, 'gleason'), (0.22962676117415212, 'pgg45')]


In [36]:
poly = PolynomialFeatures(2)

poly_linreg_model = sk.linear_model.LinearRegression()
poly_linreg_model.fit(poly.fit_transform(scaled_train_x),scaled_train_y)
predict = poly_linreg_model.predict(poly.fit_transform(scaled_test_x))
poly_linreg_error = metrics.mean_squared_error(scaled_test_y, predict)
poly_linreg_MSE = ['Polynomial Linear Regression',poly_linreg_error]

print("MSE using Polynomial Features is",poly_linreg_error)

MSE using Polynomial Features is 1.0910709612


###KNN

In [38]:
param_grid = [{'n_neighbors': [i for i in range(1,50)]}]

knn = neighbors.KNeighborsRegressor()
knn_cv = grid_search.GridSearchCV(knn, param_grid, cv=5)
knn_cv.fit(scaled_train_x, scaled_train_y)

knn_error = metrics.mean_squared_error(knn_cv.predict(scaled_test_x),scaled_test_y)
knn_MSE = ['KNN',knn_error]

print('KNN Mean Squared Error is',knn_error)
print('Best n_neighbors is',knn_cv.best_params_['n_neighbors'])

KNN Mean Squared Error is 0.576583397774
Best n_neighbors is 25


###Lasso

In [40]:
param_grid = [{'alpha': np.linspace(1e-10, 1, 50)}]

lasso_rgr = Lasso()
lasso_cv = grid_search.GridSearchCV(lasso_rgr, param_grid, cv=10)
lasso_cv.fit(scaled_train_x, scaled_train_y)
lasso_error = metrics.mean_squared_error(lasso_cv.predict(scaled_test_x),scaled_test_y)
lasso_MSE = ['Laso Linear Regression',lasso_error]

print('Lasso Mean Squared Error is',lasso_error)
print('Best alpha is',lasso_cv.best_params_['alpha'])
print('Lasso Regression Model Coefficients Are:')
print([i for i in zip(lasso_cv.best_estimator_.coef_,train_x.keys())])

Lasso Mean Squared Error is 0.441137837317
Best alpha is 0.0612244898898
Lasso Regression Model Coefficients Are:
[(0.4786536684737166, 'lcavol'), (0.19917885372928815, 'lweight'), (-0.0, 'age'), (0.10850431882662243, 'lbph'), (0.15816887459192672, 'svi'), (-0.0, 'lcp'), (0.0, 'gleason'), (0.067947279716131681, 'pgg45')]


In [41]:
param_grid = [{'alpha': np.linspace(1e-10, 1, 100)}]
mse = metrics.make_scorer(metrics.mean_squared_error, greater_is_better = False)
poly = PolynomialFeatures(3)

lasso_poly_rgr = sk.linear_model.Lasso(max_iter = 100000)
lasso_poly_cv = grid_search.GridSearchCV(lasso_poly_rgr, param_grid, cv=5, scoring=mse)
lasso_poly_cv.fit(poly.fit_transform(scaled_train_x), scaled_train_y)
Lasso_Poly_MSE = metrics.mean_squared_error(scaled_test_y, lasso_poly_cv.best_estimator_.predict(poly.fit_transform(scaled_test_x)))
poly_lasso_MSE = ['Polynomial Lasso Linear Regression',Lasso_Poly_MSE]

print('Lasso With Polynomial Features Mean Squared Error is',Lasso_Poly_MSE)
print('Best alpha is',lasso_poly_cv.best_params_['alpha'])
print('Ridge Regression Model Coefficients Are:')
print(lasso_poly_cv.best_estimator_.coef_)

Lasso With Polynomial Features Mean Squared Error is 0.39220361882
Best alpha is 0.191919192
Ridge Regression Model Coefficients Are:
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.          0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.          0.         -0.         -0.          0.
 -0.         -0.          0.          0.14452591  0.11254657  0.          0.
  0.          0.          0.          0.          0.          0.          0.
 -0.          0.          0.          0.          0.          0.          0.
 -0.          0.          0.          0.          0.          0.         -0.
  0.          0.         -0.         -0.          0.          0.          0.
  0.         -0.   

###Ridge

In [42]:
param_grid = [{'alpha': np.linspace(1e-10, 50, 100)}]

ridge_rgr = Ridge()
ridge_cv = grid_search.GridSearchCV(ridge_rgr, param_grid, cv=10)
ridge_cv.fit(scaled_train_x, scaled_train_y)
ridge_error = metrics.mean_squared_error(ridge_cv.predict(scaled_test_x),scaled_test_y)
ridge_MSE = ['Ridge Linear Regression',ridge_error]
    
print('Ridge Mean Squared Error is',ridge_error)
print('Best alpha is',ridge_cv.best_params_['alpha'])
print('Ridge Regression Model Coefficients Are:')
print([i for i in zip(ridge_cv.best_estimator_.coef_,train_x.keys())])

Ridge Mean Squared Error is 0.485482905513
Best alpha is 37.8787878788
Ridge Regression Model Coefficients Are:
[(0.30495167498017345, 'lcavol'), (0.19078884528850412, 'lweight'), (-0.017343534184502938, 'age'), (0.12444970795079691, 'lbph'), (0.17610914142925241, 'svi'), (0.037911903898169744, 'lcp'), (0.040398113553872669, 'gleason'), (0.098796194966228684, 'pgg45')]


In [43]:
param_grid = [{'alpha': np.linspace(1e-10, 50, 50)}]
mse = metrics.make_scorer(metrics.mean_squared_error, greater_is_better = False)
poly = PolynomialFeatures(2)

ridge_poly_rgr = sk.linear_model.Ridge(max_iter = 100000)
ridge_poly_cv = grid_search.GridSearchCV(ridge_poly_rgr, param_grid, cv=5, scoring=mse)
ridge_poly_cv.fit(poly.fit_transform(scaled_train_x), scaled_train_y)
ridge_Poly_MSE = metrics.mean_squared_error(scaled_test_y, ridge_poly_cv.best_estimator_.predict(poly.fit_transform(scaled_test_x)))
poly_ridge_MSE = ['Polynomial Ridge Linear Regression',ridge_Poly_MSE]

print('Ridge With Polynomial Features Mean Squared Error is',ridge_Poly_MSE)
print('Best alpha is',ridge_poly_cv.best_params_['alpha'])
print('Ridge Regression Model Coefficients Are:')
print(ridge_poly_cv.best_estimator_.coef_)

Ridge With Polynomial Features Mean Squared Error is 0.681824567937
Best alpha is 18.3673469388
Ridge Regression Model Coefficients Are:
[ 0.          0.28795119  0.17441011 -0.03704679  0.13426347  0.11503055
  0.02204136  0.08714252  0.09500646 -0.00659429 -0.05369837  0.00312567
  0.00255921  0.01421783 -0.1249906  -0.04808827  0.04225505  0.07317194
  0.00841089 -0.1492374  -0.06457997 -0.0833132  -0.09013403  0.01223538
  0.08611646 -0.0455624  -0.01178977 -0.05418748 -0.05943838  0.01798254
  0.01945335 -0.04892894 -0.02166424 -0.0263727   0.06004125  0.15239391
 -0.04819087 -0.0597749   0.08130842 -0.01375245 -0.03480112  0.05313523
 -0.01507633 -0.0091455   0.02416796]


##2) Were there any features that were highly correlated?

The strongest correlation was between 'gleason' and 'pgg45'. This suggests they are connected, possibly having similar causes that drive them or measuring a similar parameter in the patients.

##3) What features were the most important? Unimportant?

The 'lcavol' feature generally the largest coefficient, which suggests it has a greatest effect on the value of lpsa. The 'age', 'lcp', 'gleason', and 'pgg45' features all had near zero coefficients (all but 'pgg45' were eliminated by the lasso algorithm), indicating they had little effect on the value of lpsa.

##4) What model worked best? What was the MSE of each?

The best model was the lasso method using third degree polynomial features.

In [52]:
print(linreg_MSE,poly_linreg_MSE,knn_MSE,lasso_MSE,poly_lasso_MSE,ridge_MSE,poly_ridge_MSE)

['Linear Regression', 0.49112741806094645] ['Polynomial Linear Regression', 1.0910709611975311] ['KNN', 0.57658339777434686] ['Laso Linear Regression', 0.44113783731679246] ['Polynomial Lasso Linear Regression', 0.39220361881957938] ['Ridge Linear Regression', 0.48548290551339857] ['Polynomial Ridge Linear Regression', 0.68182456793670454]


##5) What difficulties did you have with this data set?

There was little variability in the 'gleason' and 'pgg45' features, so they weren't as useful as other features for developing the correlation. Also, when trying to do a lasso linear regression with a second degree polynomial transformation, the model did not converge, though a third degree polynomial transformation did work.